In this notebook, we introduce Ibis, a Python framework to access data and perform data analysis task from different sources. Basically, it allows users to perform SQL operations (select, join, filter, etc.) with Python syntax without loading data into memory. Here we perform data analysis using Ibis on nyc restuarant data as a tutorial and we use sqlite as an example back end engine.

Ibis official website: https://ibis-project.org/docs/3.2.0/#sql

Ibis introduction: https://github.com/w4111/w4111.github.io/wiki/Ibis-for-data-analysis

In [ ]:
!pip install ibis-framework 
!pip install 'ibis-framework[sqlite]'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sqlite3
import ibis
import os

## 1. Connect to SQLite database

In [4]:
# change the path to where club.db locates in drive folder
path = "/content/drive/MyDrive/Colab Notebooks/database/ibis/club.db"

ibis.options.interactive = True
db = ibis.sqlite.connect(path)

In [5]:
sql_tables = db.list_tables()
print(sql_tables)

['attends', 'budgets', 'events', 'expenses', 'fees', 'has', 'incurs', 'majors', 'members', 'pays', 'supports', 'zipcodes']


In [6]:
for each in sql_tables:
    print(f"columns in {each}: {db.table(each).columns}")

columns in attends: ['member_id', 'event_id']
columns in budgets: ['budget_id', 'category', 'spent', 'remaining', 'amount', 'event_status']
columns in events: ['event_id', 'name', 'date', 'type', 'notes', 'location', 'status']
columns in expenses: ['expense_id', 'budget_id', 'description', 'date', 'cost']
columns in fees: ['fee_id', 'date', 'amount', 'source', 'notes']
columns in has: ['event_id', 'budget_id']
columns in incurs: ['member_id', 'expense_id']
columns in majors: ['major_id', 'name', 'department', 'college']
columns in members: ['member_id', 'major_id', 'zipcode', 'firstname', 'lastname', 'email', 'position', 'tshirt_size', 'phone_number']
columns in pays: ['member_id', 'fee_id']
columns in supports: ['fee_id', 'budget_id']
columns in zipcodes: ['zipcode', 'type', 'city', 'county', 'state', 'short_state']


## 2. Basic query with ibis

Select operation

In [7]:
# select all member firstname and lastname
members = db.table(sql_tables[8])
member_name = members[["firstname", "lastname"]]
member_name

,firstname,lastname
0,Angela,Sanders
1,Grant,Gilmour
2,Luisa,Guidi
3,Randy,Woodard
4,Sacha,Harrison
5,Carlo,Jacobs
6,Phillip,Cullen
7,Emily,Jaquith
8,Tyler,Hewitt
9,Keith,Dunlop


filtering data (where condition in sql)

In [8]:
# select member_id, major_id, firstname and lastname with member_id = 5
member_info = members['member_id', 'major_id', 'firstname', 'lastname'].filter(members['member_id'] ==  5)
member_info

,member_id,major_id,firstname,lastname
0,5,2,Sacha,Harrison


ordering and limit

In [9]:
# select the description, date, cost of 5 highest expenses
expenses = db.table(sql_tables[3])
highest_exp = expenses.sort_by(ibis.desc('cost')).limit(5)['description', 'date', 'cost']
highest_exp

,description,date,cost
0,T_shirts,2019-11-19,150.00
1,Pizza,2019-08-20,122.06
2,Pizza,2019-10-22,92.82
3,Posters,2019-10-10,67.81
4,"Water, Cookies",2019-10-08,20.20


## 3. Joining and Aggregating with ibis

expressions

In [10]:
# replace each description of expenses with larger category 
expense_categoery = (expenses['description'].case()
                                        .when('Pizza', 'food')
                                        .when('Water, Cookies', 'food')
                                        .when('coke', 'food')
                                        .when('two bottles of coke', 'food')
                                        .when('Postcard', 'appliances')
                                        .when('Posters', 'appliances')
                                        .when('T_shirts', 'clothes')
                                        .else_('other')
                                        .end()
                                        .name('expense_categoery'))
expense_categoery

,expense_categoery
0,other
1,food
2,food
3,appliances
4,food
5,appliances
6,clothes
7,other
8,appliances
9,appliances


group by and join

In [11]:
# find the total budgets of each type of events
events = db.table(sql_tables[2])
has = db.table(sql_tables[5])

event_has = events.inner_join(has, predicates=events["event_id"] == has["event_id"])
event_has_budget = event_has.inner_join(expenses, predicates=expenses["budget_id"] == event_has["budget_id"])
total_budgets = event_has_budget.group_by("type").aggregate(event_has_budget["cost"].sum())

total_budgets

,type,sum
0,Election,167.09
1,Game,150.00
2,Ganme,20.20
3,Guest Speaker,70.20
4,Meeting,265.88
5,Social,24.98


In [13]:
# count the number of members from each school
majors = db.table(sql_tables[7])
member_major = members.inner_join(majors, predicates=members["major_id"] == majors["major_id"])
member_school = member_major["college"].name("member_school")
member_school_count = member_major.group_by("college").aggregate([member_school.count()])
member_school_count

,college,count
0,College of Education & Human Services,1
1,College of Humanities and Social Sciences,6
2,College of Science,2
3,Engineering,1
4,School of Business,1


## 4. Data Analysis with ibis

frequency tables

In [14]:
# compute the frequency table for the tshirt_size
members.tshirt_size.value_counts() 

,tshirt_size,count
0,Large,4
1,Medium,4
2,Small,1
3,X-Large,2


binning and histogram

In [15]:
# count the number of low, median, high expenses for all events
buckets = [0, 10, 40, 100]
bucketed = (expenses.cost
            .bucket(buckets, include_over=True)
            .name('bucket'))

bucket_counts = bucketed.value_counts()

labeled_bucket = (bucket_counts.bucket
                  .label(['0-10', '10-40', '40-100', '100+'])
                  .name('cost_range'))

expr = (bucket_counts[labeled_bucket, bucket_counts]
        .sort_by('bucket'))
expr

,cost_range,bucket,count
0,0-10,0,4
1,10-40,1,5
2,40-100,2,2
3,100+,3,2
